In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
import plotly.graph_objs as go
import glob
import unicodedata
import sklearn

In [16]:
# AWS 결측치 채우기. 결측치 index[i-1,i+1].mean() 으로 채우기
def fillAwsAllMdata(df):
        df = pd.DataFrame(df)
        count = 0
        for k in range(3,8,1):
            df_tmp = df.iloc[:,k]
            nullIndex = df_tmp.index[df.isnull().any(axis=1)].tolist()
            for i in range(len(nullIndex)):
                if i + 1 == len(nullIndex):
                    fillData = float(df.iloc[nullIndex[i]-count-1,k])
                    df.iloc[nullIndex[i]-count-1:,k] = df.iloc[nullIndex[i]-count-1:,k].fillna(fillData)
                elif nullIndex[i+1] - nullIndex[i] == 1:
                    
                    count += 1
                else:
                    if count==0:
                        fillData = float((df.iloc[nullIndex[i]-1,k]+df.iloc[nullIndex[i]+1,k])/2)
                        df.iloc[nullIndex[i],k] = fillData
                    else:
                        fillData = float((df.iloc[nullIndex[i]-count-1,k]+df.iloc[nullIndex[i]+1,k])/2)
                        df.iloc[nullIndex[i]-count-1:nullIndex[i]+1,k] = df.iloc[nullIndex[i]-count-1:nullIndex[i]+1,k].fillna(fillData)
                        
                        count = 0
        return df

In [17]:
# PM 결측치 채우기. 결측치 index[i-1,i+1].mean() 으로 채우기
def fillPmAllMdata(df):
        df = pd.DataFrame(df)
        count = 0
        k = 3
        nullIndex = df.index[df.isnull().any(axis=1)].tolist()
        for i in range(len(nullIndex)):
                if i + 1 == len(nullIndex):
                    fillData = float(df.iloc[nullIndex[i]-count-1,k])
                    df.iloc[nullIndex[i]-count-1:,k] = df.iloc[nullIndex[i]-count-1:,k].fillna(fillData)
                elif nullIndex[i+1] - nullIndex[i] == 1:
                    
                    count += 1
                else:
                    if count==0:
                        fillData = float((df.iloc[nullIndex[i]-1,k]+df.iloc[nullIndex[i]+1,k])/2)
                        df.iloc[nullIndex[i],k] = fillData
                    else:
                        if df.isnull().iloc[nullIndex[i]-count,k]:
                             fillData = df.iloc[nullIndex[i]+2,k]
                             df.iloc[:nullIndex[i]+1,k] = df.iloc[:nullIndex[i]+1,k].fillna(fillData)
                        else:
                            fillData = float((df.iloc[nullIndex[i]-count-1,k]+df.iloc[nullIndex[i]+1,k])/2)
                            df.iloc[nullIndex[i]-count-1:nullIndex[i]+1,k] = df.iloc[nullIndex[i]-count-1:nullIndex[i]+1,k].fillna(fillData)
                        
                        count = 0
        return df

In [20]:
awsPath = "TRAIN_AWS/*.csv"
pmPath = "TRAIN/*.csv"

AWS_paths = glob.glob(awsPath)
PM_paths = glob.glob(pmPath)

AWS_paths.sort()
PM_paths.sort()

conAWS_List = list()
conPM_List = list()

for i in range(len(AWS_paths)):
    AWS_paths[i] = unicodedata.normalize('NFC',AWS_paths[i])
    
# AWS 한글 지역명으로 경로값 가져오기
for i in range(len(AWS_paths)):
    globals()[f'{AWS_paths[i][10:-4]}'] = pd.read_csv(f'{AWS_paths[i]}')


# AWS 결측치 채우기
for i in range(len(AWS_paths)):
    tmp = AWS_paths[i][10:-4]
    globals()[f'{AWS_paths[i][10:-4]}_AWS'] = fillAwsAllMdata(eval(tmp))

# 지점 라벨링
for i in range(len(AWS_paths)):
    globals()[f'{AWS_paths[i][10:-4]}_AWS']['지점'] = i
    if i == 0:
        globals()[f'{AWS_paths[i][10:-4]}_AWS_CON'] = globals()[f'{AWS_paths[i][10:-4]}_AWS']
        conAWS_List.append(globals()[f'{AWS_paths[i][10:-4]}_AWS_CON'])
    elif i > 0:
        globals()[f'{AWS_paths[i][10:-4]}_AWS_CON'] = globals()[f'{AWS_paths[i][10:-4]}_AWS'].drop('일시',axis=1)
        globals()[f'{AWS_paths[i][10:-4]}_AWS_CON'] = globals()[f'{AWS_paths[i][10:-4]}_AWS_CON'].drop('연도',axis=1)
        conAWS_List.append(globals()[f'{AWS_paths[i][10:-4]}_AWS_CON'])

merged_AWS_df = pd.concat(conAWS_List,axis=1)

for i in range(len(PM_paths)):
    PM_paths[i] = unicodedata.normalize('NFC',PM_paths[i])

# PM 한글 지역명으로 경로값 가져오기
for i in range(len(PM_paths)):
    globals()[f'{PM_paths[i][6:-4]}'] = pd.read_csv(f'{PM_paths[i]}')

# PM 지역명으로 결측치 채우기
for i in range(len(PM_paths)):
    tmp = PM_paths[i][6:-4]
    globals()[f'{PM_paths[i][6:-4]}_PM'] = fillPmAllMdata(eval(tmp))

# 측정소 라벨링
for i in range(len(PM_paths)):
    globals()[f'{PM_paths[i][6:-4]}_PM']['측정소'] = i + 30
    if i == 0:
        globals()[f'{PM_paths[i][6:-4]}_PM_CON'] = globals()[f'{PM_paths[i][6:-4]}_PM']
        conPM_List.append(globals()[f'{PM_paths[i][6:-4]}_PM_CON'])
    elif i > 0:
        globals()[f'{PM_paths[i][6:-4]}_PM_CON'] = globals()[f'{PM_paths[i][6:-4]}_PM'].drop('일시',axis=1)
        globals()[f'{PM_paths[i][6:-4]}_PM_CON'] = globals()[f'{PM_paths[i][6:-4]}_PM_CON'].drop('연도',axis=1)
        conPM_List.append(globals()[f'{PM_paths[i][6:-4]}_PM_CON'])

merged_PM_df = pd.concat(conPM_List,axis=1)

In [27]:
merged_AWS_df.to_csv("AWS_TRAIN",index=False)

merged_PM_df.to_csv("PM_TRAIN",index=False)

In [23]:
merged_PM_df

,연도,일시,측정소,PM2.5,측정소,PM2.5,측정소,PM2.5,측정소,PM2.5,...,측정소,PM2.5,측정소,PM2.5,측정소,PM2.5,측정소,PM2.5,측정소,PM2.5
0,0,01-01 00:00,30,0.056,31,0.052,32,0.096,33,0.062,...,42,0.032,43,0.052,44,0.036,45,0.052,46,0.072
1,0,01-01 01:00,30,0.060,31,0.072,32,0.088,33,0.068,...,42,0.032,43,0.064,44,0.040,45,0.088,46,0.076
2,0,01-01 02:00,30,0.068,31,0.068,32,0.072,33,0.068,...,42,0.048,43,0.072,44,0.032,45,0.048,46,0.080
3,0,01-01 03:00,30,0.060,31,0.064,32,0.084,33,0.068,...,42,0.044,43,0.072,44,0.040,45,0.052,46,0.076
4,0,01-01 04:00,30,0.068,31,0.064,32,0.096,33,0.048,...,42,0.036,43,0.052,44,0.040,45,0.088,46,0.096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3,12-31 19:00,30,0.052,31,0.032,32,0.012,33,0.052,...,42,0.036,43,0.020,44,0.036,45,0.040,46,0.060
35060,3,12-31 20:00,30,0.056,31,0.048,32,0.012,33,0.048,...,42,0.036,43,0.036,44,0.060,45,0.048,46,0.052
35061,3,12-31 21:00,30,0.052,31,0.020,32,0.028,33,0.036,...,42,0.040,43,0.028,44,0.048,45,0.072,46,0.044
35062,3,12-31 22:00,30,0.052,31,0.056,32,0.024,33,0.048,...,42,0.052,43,0.036,44,0.040,45,0.060,46,0.052


,연도,일시,측정소,PM2.5
0,0,01-01 00:00,30,0.056
1,0,01-01 01:00,30,0.060
2,0,01-01 02:00,30,0.068
3,0,01-01 03:00,30,0.060
4,0,01-01 04:00,30,0.068
...,...,...,...,...
35059,3,12-31 19:00,30,0.052
35060,3,12-31 20:00,30,0.056
35061,3,12-31 21:00,30,0.052
35062,3,12-31 22:00,30,0.052
